In [1]:
!pip install --upgrade --force-reinstall torch-geometric

Looking in indexes: https://pypi.org/simple, https://nexus3.sqcorp.co/repository/pypi-all/simple
     |████████████████████████████████| 325 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 84.2 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 79.1 MB/s eta 0:00:01
     |████████████████████████████████| 38.1 MB 85.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 82.8 MB/s eta 0:00:01
     |████████████████████████████████| 23.2 MB 83.5 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 33.1 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 72.8 MB/s eta 0:00:01
     |████████████████████████████████| 407 kB 101.6 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 105.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 109.0 MB/s eta 0:00:01
     |████████████████████████████████| 596 kB 92.2 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 90.5 MB/s eta 0:00:01
   

In [22]:
import torch_geometric
print(torch_geometric.__version__)
print(torch.__version__)

2.0.2
1.9.0+cu111


In [24]:
import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import negative_sampling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
])

dataset = Planetoid(root='/tmp/Planetoid', name='Cora', transform=transform)

data = dataset[0]
print(data.to_dict())
print(data.keys)

transform = T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,)
train_data, val_data, test_data = transform(data)


{'x': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), 'edge_index': tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]], device='cuda:0'), 'y': tensor([3, 4, 4,  ..., 3, 3, 3], device='cuda:0'), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False], device='cuda:0'), 'val_mask': tensor([False, False, False,  ..., False, False, False], device='cuda:0'), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True], device='cuda:0')}
['val_mask', 'test_mask', 'edge_index', 'train_mask', 'x', 'y']


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking arugment for argument tensors in method wrapper__cat)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()




In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())



In [ ]:

best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)